# Present Pipeline

Present the transformed data

In [0]:
import dlt
from pyspark.sql import functions as F
from pyspark.sql.window import Window

source_table = spark.conf.get("source_table")
target_table = spark.conf.get("target_table")


In [0]:
catalog = "project"
br_schema = "bronze"

#Return the rules matching the tag as a format ready for DLT annotation.

def get_rules(tablename, quality):
  """
    loads data quality rules from csv file
    :param tablename: tablename to match
    :param quality: logical quality level
    :return: dictionary of rules that matched the tag
  """
  rules = {}
  df = spark.table(f"{catalog}.{br_schema}.expectations").where(f"tablename = '{tablename}' and quality ='{quality}'")
  for row in df.collect():
    rules[row['name']] = row['constraint']
  return rules

In [0]:
w = Window.orderBy("ReadingDate")


@dlt.view(name="silver_data_view", comment="Raw data from the turbines")
def silver_data_view():
    df = spark.read.table(source_table).select(
        F.col("ReadingDate"),
        F.col("TurbineNumber"),
        F.col("WindDirection"),
        F.col("PowerOutput"),
        # smooth over missing values
        F.when(
            F.col("WindSpeed").isNull(), 
            F.lag(F.col("WindSpeed")).over(w)
        ).otherwise(F.col("WindSpeed")).alias("WindSpeed")
    )
    return df


@dlt.table(name=target_table, comment="Presentation layer turbine data")
def mv_turbine_data():
    query = """
    WITH turbine_data AS
        ( 
        SELECT
            TurbineNumber,
            ReadingDate,
            WindSpeed,
            WindDirection,
            ROUND(AVG(PowerOutput) OVER (PARTITION BY TurbineNumber), 4) AS LongTermAveragePowerOutput,
            ROUND(STDDEV(PowerOutput) OVER (PARTITION BY TurbineNumber), 4) AS LongTermStandardDeviationPowerOutput,
            MIN(PowerOutput) OVER (
            PARTITION BY TurbineNumber 
            ORDER BY ReadingDate 
            RANGE BETWEEN INTERVAL 1 DAY PRECEDING AND CURRENT ROW
            ) AS MinimumPowerOutputPreviousDay,
            MAX(PowerOutput) OVER (
            PARTITION BY TurbineNumber 
            ORDER BY ReadingDate 
            RANGE BETWEEN INTERVAL 1 DAY PRECEDING AND CURRENT ROW
            ) AS MaximumPowerOutputPreviousDay,
            ROUND(AVG(PowerOutput) OVER (
            PARTITION BY TurbineNumber 
            ORDER BY ReadingDate 
            RANGE BETWEEN INTERVAL 1 DAY PRECEDING AND CURRENT ROW
            ), 2) AS AveragePowerOutputPreviousDay
        FROM 
            live.silver_data_view
        )
        SELECT 
        *,
        ROUND(ABS(AveragePowerOutputPreviousDay - LongTermAveragePowerOutput), 4) AS PowerOutputDeviationFromLongTermAverage,
        CASE 
            WHEN AveragePowerOutputPreviousDay - LongTermAveragePowerOutput > 2 * LongTermStandardDeviationPowerOutput 
            THEN 1 
            ELSE 0 
        END AS IsAnomaly 
        FROM 
        turbine_data
    """
    df = spark.sql(query)
    return df